In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

### load datasets

* Total population by country
* GDP (Current US$)
* % population who have access to electricity
* % electricity from Coal, Oil or Gas
* % electricity from Nuclear Power
* Fertility rate
* Smoking rate

In [5]:
# read csv file
df_raw = pd.read_csv('../data/ADS_project_dataset2.csv')

In [124]:
# set index and exclude rows before 1990 or after 2012 from the main data
df = df_raw[(df_raw['Year'] >= 1990) & (df_raw['Year'] <= 2012)] \
    .drop(['SmokingRate', 'EduSpend', 'RoadTrans'], axis=1) \
    .set_index(['Country', 'Year'])

### explore data

In [125]:
df.head()

,,Key,Group,AveTemperature,CO2,CO2prox1000,CO2prox2000,CO2prox3000,CO2prox4000,CO2prox5000,CO2prox6000,...,CO2prox10000,CO2prox11000,CO2prox12000,CO2prox13000,Population,GDP,FertilityRate,ElecFossi,ElecNuclear,ForestArea
Country,Year,,,,,,,,,,,,,,,,,,,,,
Afghanistan,1990,1990Afghanistan,Rest,14.993333,2.9150,183.1442,870.7274,4738.8492,3762.8294,0.0000,511.4036,...,0.0000,0.0,0.0,0.0,12249114,611392.988200,7.466,NaN,NaN,13500.0
Albania,1990,1990Albania,Rest,13.468667,6.0437,2223.4138,901.7874,0.0000,2772.0448,2428.2950,6806.8569,...,0.0000,0.0,0.0,0.0,3286542,259410.371000,2.978,13.592233,NaN,7888.0
Algeria,1990,1990Algeria,Rest,24.014417,68.0497,68.0497,2633.0391,305.8892,180.2292,3219.5983,7456.4885,...,263.6924,0.0,0.0,0.0,25912367,22325.675010,4.726,99.161699,NaN,16670.0
Andorra,1990,1990Andorra,Rest,12.455583,0.4067,2282.3870,490.4241,346.7531,0.0000,8047.9495,2896.8677,...,263.6924,0.0,0.0,0.0,54509,5959.528408,NaN,NaN,NaN,160.0
Angola,1990,1990Angola,Rest,22.466417,6.3459,6.3459,247.7112,0.0000,895.3406,2988.4974,0.0000,...,0.0000,0.0,0.0,0.0,12171441,8894.395056,7.247,13.793103,NaN,609760.0


In [126]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 4025 entries, (Afghanistan, 1990) to (Zimbabwe, 2012)
Data columns (total 23 columns):
Key               4025 non-null object
Group             4025 non-null object
AveTemperature    4025 non-null float64
CO2               3989 non-null float64
CO2prox1000       4023 non-null float64
CO2prox2000       4025 non-null float64
CO2prox3000       4025 non-null float64
CO2prox4000       4025 non-null float64
CO2prox5000       4025 non-null float64
CO2prox6000       4025 non-null float64
CO2prox7000       4025 non-null float64
CO2prox8000       4025 non-null float64
CO2prox9000       4025 non-null float64
CO2prox10000      4025 non-null float64
CO2prox11000      4025 non-null float64
CO2prox12000      4025 non-null float64
CO2prox13000      4025 non-null float64
Population        4025 non-null int64
GDP               3820 non-null float64
FertilityRate     3924 non-null float64
ElecFossi         3000 non-null float64
ElecNuclear       675 non-n

In [130]:
df.describe(include='all')

,Key,Group,AveTemperature,CO2,CO2prox1000,CO2prox2000,CO2prox3000,CO2prox4000,CO2prox5000,CO2prox6000,...,CO2prox10000,CO2prox11000,CO2prox12000,CO2prox13000,Population,GDP,FertilityRate,ElecFossi,ElecNuclear,ForestArea
count,4025,4025,4025.000000,3989.000000,4023.000000,4025.000000,4025.000000,4025.000000,4025.000000,4025.000000,...,4025.000000,4025.000000,4025.000000,4025.000000,4.025000e+03,3.820000e+03,3924.000000,3000.000000,675.000000,3.979000e+03
unique,4025,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,2009Nigeria,Rest,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,3565,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,19.264666,143.162300,728.881568,1488.895796,1715.600335,2297.327609,2909.833251,3128.833032,...,970.948211,341.045048,170.476765,33.629087,3.480574e+07,3.080441e+05,3.316458,60.579229,27.459496,2.303066e+05
std,NaN,NaN,8.615288,603.469319,1114.235958,2121.825549,2121.142422,2338.290799,3301.448303,2921.665919,...,1970.562104,1170.636752,901.629472,409.476996,1.285370e+08,1.168507e+06,1.756180,34.048486,20.941268,8.262263e+05
min,NaN,NaN,-19.000417,0.000000,0.003700,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,1.224586e+02,1.076000,0.001735,0.021151,2.200000e+00
25%,NaN,NaN,11.919417,1.773400,2.670350,142.398500,316.386500,535.039600,552.895000,889.436200,...,0.000000,0.000000,0.000000,0.000000,2.327075e+06,5.265031e+03,1.829250,32.319701,6.710160,4.774000e+03
50%,NaN,NaN,22.680750,11.230400,255.158800,523.773100,923.037500,1667.916800,1774.892900,1981.249500,...,264.607100,0.000000,0.000000,0.000000,7.759258e+06,2.176454e+04,2.764000,66.310600,26.980217,3.114000e+04
75%,NaN,NaN,26.496333,64.399300,1149.755050,1906.653600,2254.983500,2944.585200,3683.384600,5463.394900,...,724.610000,263.692400,0.000000,0.000000,2.237700e+07,1.505211e+05,4.644000,93.470967,40.618223,1.209710e+05


### Fill missing values

In [131]:
# fill missing values
df_fillna = df.copy()

In [132]:
df_fillna.ElecNuclear = df_fillna.ElecNuclear.fillna(0)

In [134]:
df_fillna = df_fillna.fillna(method='backfill')

In [146]:
df_clean = df_fillna.reset_index().drop(['Key', 'Group'], axis=1)

In [147]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4025 entries, 0 to 4024
Data columns (total 23 columns):
Country           4025 non-null object
Year              4025 non-null int64
AveTemperature    4025 non-null float64
CO2               4025 non-null float64
CO2prox1000       4025 non-null float64
CO2prox2000       4025 non-null float64
CO2prox3000       4025 non-null float64
CO2prox4000       4025 non-null float64
CO2prox5000       4025 non-null float64
CO2prox6000       4025 non-null float64
CO2prox7000       4025 non-null float64
CO2prox8000       4025 non-null float64
CO2prox9000       4025 non-null float64
CO2prox10000      4025 non-null float64
CO2prox11000      4025 non-null float64
CO2prox12000      4025 non-null float64
CO2prox13000      4025 non-null float64
Population        4025 non-null int64
GDP               4025 non-null float64
FertilityRate     4025 non-null float64
ElecFossi         4025 non-null float64
ElecNuclear       4025 non-null float64
ForestArea        

In [148]:
df_clean.reset_index().describe(include='all')

,index,Country,Year,AveTemperature,CO2,CO2prox1000,CO2prox2000,CO2prox3000,CO2prox4000,CO2prox5000,...,CO2prox10000,CO2prox11000,CO2prox12000,CO2prox13000,Population,GDP,FertilityRate,ElecFossi,ElecNuclear,ForestArea
count,4025.000000,4025,4025.000000,4025.000000,4025.000000,4025.000000,4025.000000,4025.000000,4025.000000,4025.000000,...,4025.000000,4025.000000,4025.000000,4025.000000,4.025000e+03,4.025000e+03,4025.000000,4025.000000,4025.000000,4.025000e+03
unique,NaN,175,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,Ireland,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,2012.000000,NaN,2001.000000,19.264666,142.083340,728.520844,1488.895796,1715.600335,2297.327609,2909.833251,...,970.948211,341.045048,170.476765,33.629087,3.480574e+07,2.989504e+05,3.323388,61.854565,4.605009,2.284300e+05
std,1162.061745,NaN,6.634074,8.615288,600.872095,1114.076530,2121.825549,2121.142422,2338.290799,3301.448303,...,1970.562104,1170.636752,901.629472,409.476996,1.285370e+08,1.139578e+06,1.762488,34.439878,13.368769,8.216760e+05
min,0.000000,NaN,1990.000000,-19.000417,0.000000,0.003700,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,1.224586e+02,1.076000,0.001735,0.000000,2.200000e+00
25%,1006.000000,NaN,1995.000000,11.919417,1.792200,2.670800,142.398500,316.386500,535.039600,552.895000,...,0.000000,0.000000,0.000000,0.000000,2.327075e+06,5.345388e+03,1.840000,33.407178,0.000000,4.910000e+03
50%,2012.000000,NaN,2001.000000,22.680750,11.594900,254.784800,523.773100,923.037500,1667.916800,1774.892900,...,264.607100,0.000000,0.000000,0.000000,7.759258e+06,2.212512e+04,2.762000,67.726093,0.000000,3.228600e+04
75%,3018.000000,NaN,2007.000000,26.496333,63.443400,1149.746000,1906.653600,2254.983500,2944.585200,3683.384600,...,724.610000,263.692400,0.000000,0.000000,2.237700e+07,1.568738e+05,4.652000,95.139581,0.000000,1.190820e+05


In [149]:
# split train and test
last_year = df_clean.Year.max()

In [150]:
df_train_raw = df_clean[df_clean.Year != last_year]
df_test_raw = df_clean[df_clean.Year == last_year]

In [152]:
# get X and y
y_train = df_train_raw['AveTemperature']
X_train = df_train_raw.iloc[:,3:]
y_test = df_test_raw['AveTemperature']
X_test = df_test_raw.iloc[:,3:]

In [159]:
# save datasets
y_train.to_csv('../data/y_train.csv')
X_train.to_csv('../data/X_train.csv')
y_test.to_csv('../data/y_test.csv')
X_test.to_csv('../data/X_test.csv')

### Modeling

In [154]:
# Create linear regression object
regr = linear_model.LinearRegression()

In [155]:
# Train the model using the training sets
regr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [156]:
# Make predictions using the testing set
y_pred = regr.predict(X_test)

In [157]:
# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y_test, y_pred))

Coefficients: 
 [ 3.29091918e-04 -2.52426690e-03 -2.30945960e-04  1.73165734e-04
 -8.66124443e-05 -2.86949232e-05  2.78828657e-05  1.70548383e-04
  8.20898394e-04  8.33917011e-04  1.05249699e-03  1.02436973e-03
 -1.61836583e-04 -2.02029374e-03  4.52291794e-09  1.12848297e-07
  1.25867591e+00  4.74189755e-02  9.17316219e-03 -1.64135276e-06]
Mean squared error: 33.08
Variance score: 0.56


In [162]:
import statsmodels.api as sm
X = sm.add_constant(X_train)
model = sm.OLS(y_train, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         AveTemperature   R-squared:                       0.651
Model:                            OLS   Adj. R-squared:                  0.650
Method:                 Least Squares   F-statistic:                     357.7
Date:                Sun, 30 Sep 2018   Prob (F-statistic):               0.00
Time:                        16:14:30   Log-Likelihood:                -11725.
No. Observations:                3850   AIC:                         2.349e+04
Df Residuals:                    3829   BIC:                         2.362e+04
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             9.7012      0.646     15.012      0.000       8.434      10.968
CO2               0.0003      0.000      1.373      0.170      -0.000       0.001
CO2prox1000      -0.0025      0.000    -21.041      0.000      -0.003      -0.002
CO2prox2000      -0.0002   5.16e-05     -4.473      0.000      -0.000      -0.000
CO2prox3000       0.0002   4.84e-05      3.575      0.000    7.82e-05       0.000
CO2prox4000   -8.661e-05   4.34e-05     -1.994      0.046      -0.000   -1.45e-06
CO2prox5000   -2.869e-05    3.7e-05     -0.775      0.438      -0.000    4.39e-05
CO2prox6000    2.788e-05   3.83e-05      0.728      0.467   -4.72e-05       0.000
CO2prox7000       0.0002   3.57e-05      4.772      0.000       0.000       0.000
CO2prox8000       0.0008   4.82e-05     17.035      0.000       0.001       0.001
CO2prox9000       0.0008   4.39e-05     18.988      0.000       0.001       0.001
CO2prox10000      0.0011   5.14e-05     20.486      0.000       0.001       0.001
CO2prox11000      0.0010   8.44e-05     12.130      0.000       0.001       0.001
CO2prox12000     -0.0002      0.000     -1.574      0.116      -0.000    3.98e-05
CO2prox13000     -0.0020      0.000     -9.085      0.000      -0.002      -0.002
Population     4.523e-09   8.83e-10      5.124      0.000    2.79e-09    6.25e-09
GDP            1.128e-07   7.51e-08      1.502      0.133   -3.45e-08     2.6e-07
FertilityRate     1.2587      0.065     19.386      0.000       1.131       1.386
ElecFossi         0.0474      0.003     18.192      0.000       0.042       0.053
ElecNuclear       0.0092      0.007      1.246      0.213      -0.005       0.024
ForestArea    -1.641e-06    1.2e-07    -13.690      0.000   -1.88e-06   -1.41e-06
==============================================================================
Omnibus:                     1081.330   Durbin-Watson:                   2.129
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4411.881
Skew:                          -1.326   Prob(JB):                         0.00
Kurtosis:                       7.525   Cond. No.                     1.04e+09
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.04e+09. This might indicate that there are
strong multicollinearity or other numerical problems.
"""